In [1]:
import sys
print(sys.executable)

/home/mateo/.pyenv/versions/3.12.8/bin/python


In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import langid
import time
import ast
import timeit

#!pip install transformers torch --break-system-packages
from transformers import pipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [14]:
movies_sample_path = os.path.abspath("../.data/movies_sample.csv")
raitings_path = os.path.abspath("../.data/ratings_sample.csv")
credits_path = os.path.abspath("../.data/credits_sample.csv")
movies_df = pd.read_csv(movies_sample_path, low_memory=False)
ratings_df = pd.read_csv(raitings_path)
credits_df = pd.read_csv(credits_path)

In [15]:
movies_df.head()

,id,title,genres,release_date,overview,production_countries,spoken_languages,budget,revenue
0,24621,Chasing Papi,"['Comedy', 'Romance']",2003-04-16,Playboy Thomas Fuentes has so far been able to skate by in life on good looks and charm alone. B...,['United States of America'],['English'],0,6126237.0
1,48118,The Cold Light of Day,"['Drama', 'Thriller']",1996-08-20,A troubled detective befriends a single woman and her daughter with the intention of using them ...,"['Germany', 'Netherlands', 'United Kingdom']",['English'],0,0.0
2,9503,Pope Joan,['Drama'],2009-10-22,A 9th century woman of English extraction born in the German city of Ingelheim disguises herself...,"['Spain', 'Germany', 'United Kingdom', 'Italy']","['English', 'Latin']",0,27412220.0
3,436932,Jawbone,"['Action', 'Drama']",2017-05-12,"A former youth boxing champion, Jimmy McCabe is a man in search of hope but looking in all the w...",['United Kingdom'],['English'],0,0.0
4,125531,Pripyat,['Documentary'],1999-09-11,The documentary covers the stories of the people that choose to work and live in Pripyat. They t...,['Austria'],['Pусский'],0,0.0


In [17]:
movies_df.dtypes

id                        int64
title                    object
genres                   object
release_date             object
overview                 object
production_countries     object
spoken_languages         object
budget                    int64
revenue                 float64
dtype: object

## Ejemplo resolucion Query1: Peliculas y sus géneros de los años 00' con producción Argentina y Española